# 🔄 **04 - Pipeline: Integração Completa**

## 🎯 **Objetivo:**
Integrar todos os componentes (Embeddings + RAG + LangGraph) em um pipeline unificado.

## 📋 **O que faremos:**
1. 🔄 Importar componentes das etapas anteriores
2. 🏗️ Criar pipeline integrado
3. 🧪 Testes de integração completa
4. 🚀 Otimizações de performance

---

## 1️⃣ **Importação dos Componentes**

In [1]:
# 📦 IMPORTAÇÕES E SETUP

import sys
import os
import json
import numpy as np
from pathlib import Path
from typing import Dict, List, Any, Optional, TypedDict
import warnings
warnings.filterwarnings('ignore')

# Verificar se estamos no diretório correto
current_dir = Path.cwd()
print(f"📂 Diretório atual: {current_dir}")

# Importações específicas
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    print("✅ Embeddings: OK")
except ImportError as e:
    print(f"❌ Erro embeddings: {e}")

try:
    from langgraph.graph import StateGraph, END
    from langgraph.checkpoint.memory import MemorySaver
    print("✅ LangGraph: OK")
except ImportError as e:
    print(f"❌ Erro LangGraph: {e}")

📂 Diretório atual: /home/kevin/Documentos/Rede_Neural_Artificial/lang_graph/Trabalho_Prático_03
✅ Embeddings: OK
✅ Embeddings: OK
✅ LangGraph: OK
✅ LangGraph: OK


## 2️⃣ **Carregar Componentes das Etapas Anteriores**

In [2]:
# 🧮 SISTEMA DE EMBEDDINGS (da Etapa 01)

class EmbeddingSystem:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.chunks = []
        self.embeddings = None
        
    def load_chunks(self, chunks_file='fitness_chunks.json'):
        """Carrega chunks processados"""
        try:
            with open(chunks_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.chunks = data.get('chunks', [])
                print(f"📄 Carregados {len(self.chunks)} chunks")
                return True
        except FileNotFoundError:
            print(f"❌ Arquivo {chunks_file} não encontrado")
            return False
            
    def generate_embeddings(self):
        """Gera embeddings dos chunks"""
        if not self.chunks:
            print("❌ Nenhum chunk carregado")
            return
            
        textos = [chunk['texto'] for chunk in self.chunks]
        self.embeddings = self.model.encode(textos)
        print(f"🧮 Embeddings gerados: {self.embeddings.shape}")
        
    def search_similar(self, query, top_k=3):
        """Busca chunks similares"""
        if self.embeddings is None:
            return []
            
        query_embedding = self.model.encode([query])
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Top-k indices
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            results.append({
                'chunk': self.chunks[idx],
                'similarity': float(similarities[idx])
            })
            
        return results

# Inicializar sistema
embedding_system = EmbeddingSystem()
embedding_system.load_chunks()
embedding_system.generate_embeddings()
print("🧮 Sistema de Embeddings: Carregado")

📄 Carregados 22 chunks
🧮 Embeddings gerados: (22, 384)
🧮 Sistema de Embeddings: Carregado
🧮 Embeddings gerados: (22, 384)
🧮 Sistema de Embeddings: Carregado


In [3]:
# 🧠 SISTEMA RAG (da Etapa 02)

class FitnessRAGSystem:
    def __init__(self, embedding_system):
        self.embedding_system = embedding_system
        self.cache = {}
        
    def get_context(self, objetivo, periodicidade, experiencia="intermediário"):
        """Busca contexto específico"""
        cache_key = f"{objetivo}_{periodicidade}_{experiencia}"
        
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        # Queries específicas
        queries = [
            f"exercícios para {objetivo}",
            f"treino {periodicidade} vezes por semana",
            f"programa {experiencia}",
            "princípios treinamento"
        ]
        
        contexto = {}
        for query in queries:
            results = self.embedding_system.search_similar(query, top_k=2)
            contexto[query] = results
            
        self.cache[cache_key] = contexto
        return contexto
        
    def extract_exercises(self, contexto):
        """Extrai exercícios do contexto"""
        exercises = set()
        
        for query, results in contexto.items():
            for result in results:
                texto = result['chunk']['texto'].lower()
                
                # Lista de exercícios conhecidos
                exercicios_conhecidos = [
                    'supino', 'agachamento', 'levantamento terra', 'desenvolvimento',
                    'puxada', 'remada', 'rosca direta', 'tríceps pulley',
                    'leg press', 'cadeira extensora', 'mesa flexora'
                ]
                
                for ex in exercicios_conhecidos:
                    if ex in texto:
                        exercises.add(ex)
                        
        return list(exercises)

# Inicializar RAG
rag_system = FitnessRAGSystem(embedding_system)
print("🧠 Sistema RAG: Carregado")

🧠 Sistema RAG: Carregado


## 3️⃣ **Estado e Nós LangGraph (da Etapa 03)**

In [4]:
# 🏗️ ESTADO DO PIPELINE

class FitnessState(TypedDict):
    # Entrada do usuário
    pergunta: str
    idade: Optional[int]
    peso: Optional[float]
    objetivo: Optional[str]
    periodicidade: Optional[int]
    experiencia: Optional[str]
    
    # Cálculos
    imc: Optional[float]
    calorias_dia: Optional[int]
    
    # Contextos
    contexto_rag: Optional[Dict]
    contexto_web: Optional[str]
    exercicios: Optional[List[str]]
    
    # Saída
    plano_treino: Optional[str]
    resposta_final: Optional[str]
    
    # Controle
    etapa_atual: str
    status: str

print("🏗️ Estado FitnessState: Definido")

🏗️ Estado FitnessState: Definido


In [5]:
# 🔧 NÓS DO PIPELINE

def no_validacao(state: FitnessState) -> FitnessState:
    """Valida e extrai dados da pergunta"""
    pergunta = state.get('pergunta', '').lower()
    
    # Extrair dados básicos (simulado)
    idade = 25  # Padrão
    peso = 70.0  # Padrão
    periodicidade = 3  # Padrão
    
    # Detectar objetivo
    if any(word in pergunta for word in ['hipertrofia', 'massa', 'muscular']):
        objetivo = 'hipertrofia'
    elif any(word in pergunta for word in ['emagrecimento', 'perder peso', 'definição']):
        objetivo = 'emagrecimento'
    else:
        objetivo = 'condicionamento'
        
    # Detectar experiência
    if 'iniciante' in pergunta:
        experiencia = 'iniciante'
    elif 'avançado' in pergunta:
        experiencia = 'avançado'
    else:
        experiencia = 'intermediário'
    
    return {
        **state,
        'idade': idade,
        'peso': peso,
        'objetivo': objetivo,
        'periodicidade': periodicidade,
        'experiencia': experiencia,
        'etapa_atual': 'validacao',
        'status': 'dados_validados'
    }

def no_calculos(state: FitnessState) -> FitnessState:
    """Realiza cálculos básicos"""
    peso = state.get('peso', 70)
    idade = state.get('idade', 25)
    
    # Cálculo IMC
    altura_padrao = 1.75  # Assumindo altura padrão
    imc = peso / (altura_padrao ** 2)
    
    # Cálculo calorias (TMB + atividade)
    tmb = 88.362 + (13.397 * peso) + (4.799 * altura_padrao * 100) - (5.677 * idade)
    calorias_dia = int(tmb * 1.5)  # Ativo
    
    return {
        **state,
        'imc': round(imc, 1),
        'calorias_dia': calorias_dia,
        'etapa_atual': 'calculos',
        'status': 'calculos_realizados'
    }

def no_rag(state: FitnessState) -> FitnessState:
    """Busca contexto na base de conhecimento"""
    objetivo = state.get('objetivo', 'hipertrofia')
    periodicidade = state.get('periodicidade', 3)
    experiencia = state.get('experiencia', 'intermediário')
    
    # Buscar contexto RAG
    contexto_rag = rag_system.get_context(objetivo, periodicidade, experiencia)
    exercicios = rag_system.extract_exercises(contexto_rag)
    
    return {
        **state,
        'contexto_rag': contexto_rag,
        'exercicios': exercicios[:8],  # Limitar a 8 exercícios
        'etapa_atual': 'rag',
        'status': 'contexto_recuperado'
    }

def no_geracao(state: FitnessState) -> FitnessState:
    """Gera o plano de treino final"""
    objetivo = state.get('objetivo', 'hipertrofia')
    periodicidade = state.get('periodicidade', 3)
    experiencia = state.get('experiencia', 'intermediário')
    exercicios = state.get('exercicios', [])
    imc = state.get('imc', 0)
    calorias = state.get('calorias_dia', 0)
    
    # Gerar plano baseado nos dados
    plano = f"""
# 🏋️‍♂️ PLANO DE TREINO PERSONALIZADO

## 👤 PERFIL:
- Objetivo: {objetivo.upper()}
- Experiência: {experiencia.title()}
- Frequência: {periodicidade}x por semana
- IMC: {imc}
- Calorias/dia: {calorias}

## 💪 EXERCÍCIOS RECOMENDADOS:
{chr(10).join(f'• {ex.title()}' for ex in exercicios)}

## 📋 ESTRUTURA SEMANAL:
**Treino A - Superior:**
1. Supino reto - 4x8-12
2. Puxada frontal - 4x8-12
3. Desenvolvimento - 3x10-15
4. Rosca direta - 3x10-15

**Treino B - Inferior:**
1. Agachamento - 4x8-12
2. Levantamento terra - 4x6-10
3. Leg press - 3x12-15
4. Panturrilha - 4x15-20

**Treino C - Full Body:**
1. Supino inclinado - 3x10-12
2. Remada curvada - 3x10-12
3. Agachamento frontal - 3x10-12
4. Abdominais - 3x15-20
"""
    
    resposta = f"""
✅ **Plano criado com sucesso!**

🎯 Seu treino foi personalizado para **{objetivo}** com base na sua experiência **{experiencia}** e frequência de **{periodicidade}x por semana**.

📊 **Dados calculados:**
- IMC: {imc} 
- Necessidade calórica: {calorias} cal/dia

💪 **{len(exercicios)} exercícios** foram selecionados da base de conhecimento fitness.

🔄 Siga a progressão gradual e ajuste as cargas conforme sua evolução!
"""
    
    return {
        **state,
        'plano_treino': plano,
        'resposta_final': resposta,
        'etapa_atual': 'geracao',
        'status': 'completo'
    }

print("🔧 Nós do Pipeline: Definidos")

🔧 Nós do Pipeline: Definidos


## 4️⃣ **Montagem do Pipeline Integrado**

In [6]:
# 🌐 PIPELINE LANGGRAPH INTEGRADO

def criar_pipeline_fitness():
    """Cria o pipeline completo integrado"""
    
    # Criar grafo
    workflow = StateGraph(FitnessState)
    
    # Adicionar nós
    workflow.add_node("validacao", no_validacao)
    workflow.add_node("calculos", no_calculos)
    workflow.add_node("rag", no_rag)
    workflow.add_node("geracao", no_geracao)
    
    # Definir fluxo
    workflow.set_entry_point("validacao")
    workflow.add_edge("validacao", "calculos")
    workflow.add_edge("calculos", "rag")
    workflow.add_edge("rag", "geracao")
    workflow.add_edge("geracao", END)
    
    # Compilar com memória
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

# Criar pipeline
pipeline_fitness = criar_pipeline_fitness()
print("🌐 Pipeline Fitness: Criado e Compilado")
print("🔄 Fluxo: Validação → Cálculos → RAG → Geração")

🌐 Pipeline Fitness: Criado e Compilado
🔄 Fluxo: Validação → Cálculos → RAG → Geração


## 5️⃣ **Testes de Integração Completa**

In [7]:
# 🧪 FUNÇÃO DE TESTE INTEGRADA

def testar_pipeline_completo(pergunta, thread_id="test-1"):
    """Testa o pipeline completo"""
    print(f"\n🧪 TESTE: {pergunta}")
    print("=" * 60)
    
    # Estado inicial
    estado_inicial = {
        "pergunta": pergunta,
        "etapa_atual": "inicio",
        "status": "iniciado"
    }
    
    # Configuração thread
    config = {"configurable": {"thread_id": thread_id}}
    
    # Executar pipeline
    try:
        resultado = pipeline_fitness.invoke(estado_inicial, config)
        
        print("\n📊 RESULTADO DA EXECUÇÃO:")
        print(f"✅ Status: {resultado.get('status')}")
        print(f"🎯 Objetivo: {resultado.get('objetivo')}")
        print(f"🏃 Experiência: {resultado.get('experiencia')}")
        print(f"📅 Periodicidade: {resultado.get('periodicidade')}x/semana")
        print(f"📏 IMC: {resultado.get('imc')}")
        print(f"🔥 Calorias: {resultado.get('calorias_dia')}")
        print(f"💪 Exercícios: {len(resultado.get('exercicios', []))}")
        
        print("\n📝 RESPOSTA GERADA:")
        print(resultado.get('resposta_final', 'Nenhuma resposta gerada'))
        
        return resultado
        
    except Exception as e:
        print(f"❌ Erro na execução: {e}")
        return None

print("🧪 Função de Teste: Pronta")

🧪 Função de Teste: Pronta


In [8]:
# 🚀 EXECUTAR TESTES

cenarios_teste = [
    "Quero um treino para hipertrofia, sou iniciante",
    "Preciso de um plano para emagrecimento, nível intermediário",
    "Treino de condicionamento físico para avançado"
]

resultados_testes = []

for i, cenario in enumerate(cenarios_teste, 1):
    resultado = testar_pipeline_completo(cenario, f"test-{i}")
    if resultado:
        resultados_testes.append({
            'cenario': cenario,
            'status': resultado.get('status'),
            'objetivo': resultado.get('objetivo'),
            'exercicios_count': len(resultado.get('exercicios', []))
        })
    print("\n" + "="*80 + "\n")

print("\n📊 RESUMO DOS TESTES:")
for i, resultado in enumerate(resultados_testes, 1):
    print(f"{i}. {resultado['cenario'][:30]}... → {resultado['status']} ({resultado['exercicios_count']} exercícios)")


🧪 TESTE: Quero um treino para hipertrofia, sou iniciante

📊 RESULTADO DA EXECUÇÃO:
✅ Status: completo
🎯 Objetivo: hipertrofia
🏃 Experiência: iniciante
📅 Periodicidade: 3x/semana
📏 IMC: 22.9
🔥 Calorias: 2586
💪 Exercícios: 6

📝 RESPOSTA GERADA:

✅ **Plano criado com sucesso!**

🎯 Seu treino foi personalizado para **hipertrofia** com base na sua experiência **iniciante** e frequência de **3x por semana**.

📊 **Dados calculados:**
- IMC: 22.9 
- Necessidade calórica: 2586 cal/dia

💪 **6 exercícios** foram selecionados da base de conhecimento fitness.

🔄 Siga a progressão gradual e ajuste as cargas conforme sua evolução!




🧪 TESTE: Preciso de um plano para emagrecimento, nível intermediário

📊 RESULTADO DA EXECUÇÃO:
✅ Status: completo
🎯 Objetivo: emagrecimento
🏃 Experiência: intermediário
📅 Periodicidade: 3x/semana
📏 IMC: 22.9
🔥 Calorias: 2586
💪 Exercícios: 6

📝 RESPOSTA GERADA:

✅ **Plano criado com sucesso!**

🎯 Seu treino foi personalizado para **emagrecimento** com base na sua experi

## 6️⃣ **Pipeline Otimizado para Uso**

In [9]:
# 🎯 CLASSE PIPELINE OTIMIZADA

class FitnessPipelineCompleto:
    def __init__(self):
        self.embedding_system = EmbeddingSystem()
        self.rag_system = None
        self.pipeline = None
        self._inicializado = False
        
    def inicializar(self):
        """Inicializa todos os componentes"""
        if self._inicializado:
            return True
            
        try:
            # Carregar embeddings
            if not self.embedding_system.load_chunks():
                return False
            self.embedding_system.generate_embeddings()
            
            # Inicializar RAG
            self.rag_system = FitnessRAGSystem(self.embedding_system)
            
            # Criar pipeline
            self.pipeline = criar_pipeline_fitness()
            
            self._inicializado = True
            print("✅ Pipeline completo inicializado com sucesso!")
            return True
            
        except Exception as e:
            print(f"❌ Erro na inicialização: {e}")
            return False
    
    def gerar_treino(self, pergunta, thread_id=None):
        """Gera treino personalizado"""
        if not self._inicializado:
            print("❌ Pipeline não inicializado. Use .inicializar() primeiro.")
            return None
            
        if thread_id is None:
            import time
            thread_id = f"user-{int(time.time())}"
            
        estado_inicial = {
            "pergunta": pergunta,
            "etapa_atual": "inicio",
            "status": "iniciado"
        }
        
        config = {"configurable": {"thread_id": thread_id}}
        
        try:
            resultado = self.pipeline.invoke(estado_inicial, config)
            return {
                'sucesso': True,
                'plano_treino': resultado.get('plano_treino'),
                'resposta': resultado.get('resposta_final'),
                'dados': {
                    'objetivo': resultado.get('objetivo'),
                    'experiencia': resultado.get('experiencia'),
                    'periodicidade': resultado.get('periodicidade'),
                    'imc': resultado.get('imc'),
                    'calorias': resultado.get('calorias_dia'),
                    'exercicios': resultado.get('exercicios', [])
                }
            }
        except Exception as e:
            return {
                'sucesso': False,
                'erro': str(e)
            }

# Criar instância global
fitness_coach = FitnessPipelineCompleto()
print("🎯 Classe Pipeline Completa: Criada")

🎯 Classe Pipeline Completa: Criada


In [10]:
# 🚀 DEMONSTRAÇÃO FINAL

# Inicializar
if fitness_coach.inicializar():
    print("\n🎯 SISTEMA PRONTO PARA USO!\n")
    
    # Teste final
    pergunta_demo = "Quero um treino para hipertrofia, sou intermediário e posso treinar 4 vezes por semana"
    
    print(f"💬 Pergunta: {pergunta_demo}")
    print("\n🔄 Processando...")
    
    resultado = fitness_coach.gerar_treino(pergunta_demo)
    
    if resultado['sucesso']:
        print("\n" + "="*60)
        print(resultado['resposta'])
        print("\n📋 PLANO COMPLETO:")
        print(resultado['plano_treino'][:500] + "...")
        print("\n✅ PIPELINE INTEGRADO FUNCIONANDO PERFEITAMENTE!")
    else:
        print(f"❌ Erro: {resultado['erro']}")
else:
    print("❌ Falha na inicialização do pipeline")

📄 Carregados 22 chunks
🧮 Embeddings gerados: (22, 384)
✅ Pipeline completo inicializado com sucesso!

🎯 SISTEMA PRONTO PARA USO!

💬 Pergunta: Quero um treino para hipertrofia, sou intermediário e posso treinar 4 vezes por semana

🔄 Processando...


✅ **Plano criado com sucesso!**

🎯 Seu treino foi personalizado para **hipertrofia** com base na sua experiência **intermediário** e frequência de **3x por semana**.

📊 **Dados calculados:**
- IMC: 22.9 
- Necessidade calórica: 2586 cal/dia

💪 **6 exercícios** foram selecionados da base de conhecimento fitness.

🔄 Siga a progressão gradual e ajuste as cargas conforme sua evolução!


📋 PLANO COMPLETO:

# 🏋️‍♂️ PLANO DE TREINO PERSONALIZADO

## 👤 PERFIL:
- Objetivo: HIPERTROFIA
- Experiência: Intermediário
- Frequência: 3x por semana
- IMC: 22.9
- Calorias/dia: 2586

## 💪 EXERCÍCIOS RECOMENDADOS:
• Remada
• Supino
• Agachamento
• Leg Press
• Desenvolvimento
• Puxada

## 📋 ESTRUTURA SEMANAL:
**Treino A - Superior:**
1. Supino reto - 4x8-12
2. P

## 📊 **Resumo da Etapa 04**

### ✅ **Conquistas:**
- 🔄 Integração completa de todos os componentes (Embeddings + RAG + LangGraph)
- 🏗️ Pipeline unificado e otimizado
- 🧪 Testes de integração abrangentes
- 🎯 Classe wrapper para facilitar uso
- 📊 Monitoramento completo do fluxo

### 🌐 **Arquitetura Final:**
```
Pergunta → Validação → Cálculos → RAG → Geração → Resposta
           ↓           ↓          ↓      ↓
        Extração   Matemática   Base   Plano
         Dados      IMC/Cal    Fitness Completo
```

### 🎯 **Componentes Integrados:**
1. **Sistema de Embeddings** - Busca semântica na base fitness
2. **Sistema RAG** - Recuperação contextual de exercícios
3. **Pipeline LangGraph** - Fluxo estruturado e confiável
4. **Cálculos Automáticos** - IMC, calorias e personalizações
5. **Geração Inteligente** - Planos completos personalizados

### 📈 **Próximo Passo:**
- **05-Sistema.ipynb**: Interface final e demonstrações completas

---

🔄 **Pipeline de IA Fitness totalmente integrado e funcional!** 🏋️‍♂️